# Tests

In [1]:
import os
import sys
import logging
import json
import re
import pandas as pd

#dir = os.path.abspath(__file__)
#dir    = os.getcwd()
#pkgdir = os.path.dirname(os.path.dirname(dir)

sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))
from lib.nanuq import Nanuq
from lib.gapp import Phenotips
from lib.gapp import BSSH

nq = Nanuq()
#pho = Phenotips()
bssh = BSSH()

In [2]:
class Args:
    def __init__(self, run='A00516_0445', level='info'):
        self.run   = run
        self.level = level

args = Args(level='debug')
vars(args)

{'run': 'A00516_0445', 'level': 'debug'}

In [3]:
os.chdir("D:\\HSJ\\Projects\\PRAGMatIQ\\Runs")
#os.listdir()

In [4]:
file = "A00516_0445\\emg_batch_manifest.csv"
def get_samples_list(file):
    samples = []
    with open(file, 'r') as fh:
        for line in fh.readlines():
            cols = line.split(',')
            samples.append(cols[5])
    samples.pop(0)
    return samples
print(get_samples_list(file))


['GM231651', 'GM231626', 'GM231624', 'GM231632', 'GM231627', 'GM231615', 'GM231667', 'GM231668', '23-05982-T1', '23-06384-T1', '23-06383-T1', '3042652455', '3042642360', '3042645886', '3052768938', '3052768961', '3052768951', '3052768942']


In [5]:
nq = Nanuq()
samplenames = nq.get_samplenames("A00516_0445")
samplenames.text

'##2023-08-09\r\n##Centre for Pediatric Clinical Genomics\r\n##Flow Cell: H7N33DSX7\r\n##Principal Investigator: Dr Mehdi Yeganeh, CHUS Service de génétique médicale, Dr Jean-François Soucy\r\n##Nanuq References: A00516_0445\r\n##Content: Internal_Sample_ID -> Client_Sample_Name Conversion grid\r\n##-------------------------------------------\r\n##Internal_Sample_ID\tClient_Sample_Name\r\n##-------------------------------------------\r\n22256\t3042652455\r\n22257\t3042642360\r\n22258\t3042645886\r\n22262\t3052768938\r\n22263\t3052768942\r\n22264\t3052768961\r\n22265\t3052768951\r\n22282\t23-05982-T1\r\n22283\t23-06383-T1\r\n22284\t23-06384-T1\r\n22285\tGM231615\r\n22286\tGM231624\r\n22287\tGM231626\r\n22288\tGM231627\r\n22290\tGM231632\r\n22293\tGM231651\r\n22294\tGM231667\r\n22295\tGM231668\r\n##-------------------------------------------\r\n##Description of the conversion grid:\r\n##A "Client_Sample_Name" represents the name of a sequencing sample initially assigned by the client.\r\

In [6]:
for line in samplenames.text.splitlines():
    print(line)

##2023-08-09
##Centre for Pediatric Clinical Genomics
##Flow Cell: H7N33DSX7
##Principal Investigator: Dr Mehdi Yeganeh, CHUS Service de génétique médicale, Dr Jean-François Soucy
##Nanuq References: A00516_0445
##Content: Internal_Sample_ID -> Client_Sample_Name Conversion grid
##-------------------------------------------
##Internal_Sample_ID	Client_Sample_Name
##-------------------------------------------
22256	3042652455
22257	3042642360
22258	3042645886
22262	3052768938
22263	3052768942
22264	3052768961
22265	3052768951
22282	23-05982-T1
22283	23-06383-T1
22284	23-06384-T1
22285	GM231615
22286	GM231624
22287	GM231626
22288	GM231627
22290	GM231632
22293	GM231651
22294	GM231667
22295	GM231668
##-------------------------------------------
##Description of the conversion grid:
##A "Client_Sample_Name" represents the name of a sequencing sample initially assigned by the client.
##The "Internal_Sample_ID" is the internal identifier assigned by the Center for this sample.
##Use of an "In

In [7]:
fc_date = re.match(r'##(\d{4}-\d{2}-\d{2})', samplenames.text).group(1)
fc_date

'2023-08-09'

In [8]:
for line in samplenames.text.splitlines():
    if not line.startswith('#'):
        cqgc, sample = line.split("\t")
        print(cqgc, sample)


22256 3042652455
22257 3042642360
22258 3042645886
22262 3052768938
22263 3052768942
22264 3052768961
22265 3052768951
22282 23-05982-T1
22283 23-06383-T1
22284 23-06384-T1
22285 GM231615
22286 GM231624
22287 GM231626
22288 GM231627
22290 GM231632
22293 GM231651
22294 GM231667
22295 GM231668


In [9]:
json.loads(nq.get_sample(22283))

[{'ldmSampleId': '23-06383-T1',
  'ldm': 'LDM-CHUS',
  'patient': {'designFamily': 'TRIO',
   'familyId': '23-05982-T1',
   'familyMember': 'FTH',
   'firstName': 'Andre-Philippe',
   'lastName': 'Belley',
   'ramq': 'BELA93092213',
   'sex': 'MALE',
   'mrn': '00000000',
   'ep': 'CHUS',
   'birthDate': '22/09/1993',
   'fetus': False,
   'status': 'UNK'},
  'ldmServiceRequestId': '23-06383-T1',
  'labAliquotId': '22283',
  'panelCode': 'PRAGMATIQ',
  'specimenType': 'NBL',
  'sampleType': 'DNA',
  'ldmSpecimenId': '23-06383-T1'}]

In [10]:
os.getcwd()

'D:\\HSJ\\Projects\\PRAGMatIQ\\Runs'

In [12]:
samples_list = f"{args.run}{os.sep}samples_list.txt"
with open(samples_list, 'r') as fh:
    next(fh) # skip the header
    for line in fh.readlines():
        sample = line.split(',')[0]
        print(f"aws subprocess for {sample}")

aws subprocess for GM231651
aws subprocess for GM231626
aws subprocess for GM231624
aws subprocess for GM231632
aws subprocess for GM231627
aws subprocess for GM231615
aws subprocess for GM231667
aws subprocess for GM231668
aws subprocess for 23-05982-T1
aws subprocess for 23-06384-T1
aws subprocess for 23-06383-T1
aws subprocess for 3042652455
aws subprocess for 3042642360
aws subprocess for 3042645886
aws subprocess for 3052768938
aws subprocess for 3052768961
aws subprocess for 3052768951
aws subprocess for 3052768942


In [13]:
names = ['hien', 'caro', 'esm', 'sts']
enumerate(names)

In [14]:
for index, name in enumerate(names):
    print(index, name)

ValueError: too many values to unpack (expected 2)